DataAdjust.py
3. Drop rows with any N/A values.
4. Group the data by 'Symbol'.
5. For each group, create a summary row containing:
    - Symbol
    - Start Date
    - End Date
    - Original Adjusted Close
    - Final Adjusted Close
    - Original Earnings
    - Final Earnings
    - Original P/E Ratio
    - Final P/E Ratio
6. Write the summary data to 'stocks_summary_one_row.csv'.

In [ ]:
import pandas as pd
# Read the CSV
df = pd.read_csv("stocks_calc_final.csv")

# Ensure Date is recognized as a datetime for proper sorting
df["Date"] = pd.to_datetime(df["Date"], errors="coerce")

# Drop rows with any N/A values
df_clean = df.dropna()

# Group by symbol
grouped = df_clean.groupby("Symbol", group_keys=True)

summary_rows = []
for symbol, group in grouped:
    # Sort by Date so the first row is the earliest date, the last row is the latest date
    group_sorted = group.sort_values("Date")
    first_row = group_sorted.iloc[0]
    last_row = group_sorted.iloc[-1]
    
    # Create a single summary row for this symbol
    summary_rows.append({
        "Symbol": symbol,
        "Start Date": first_row["Date"].strftime("%Y-%m-%d"),
        "End Date": last_row["Date"].strftime("%Y-%m-%d"),
        "Original Adjusted Close": first_row["Adjusted Close"],
        "Final Adjusted Close": last_row["Adjusted Close"],
        "Original Earnings": first_row["Earnings"],
        "Final Earnings": last_row["Earnings"],
        "Original P/E Ratio": first_row["P/E Ratio"],
        "Final P/E Ratio": last_row["P/E Ratio"]
    })

# Create a new DataFrame and write to CSV
summary_df = pd.DataFrame(summary_rows)
summary_df.to_csv("stocks_summary_one_row.csv", index=False)

This script processes stock data from a CSV file, calculates trailing twelve months (TTM) earnings per share (EPS) 
and price-to-earnings (P/E) ratios, and outputs the results to a new CSV file.
Steps:
1. Read stock data from 'stocks_calc_final.csv'.
2. Convert the 'Date' column to datetime format and sort the data by 'Symbol' and 'Date'.
3. Group the data by 'Symbol'.
4. For each group, calculate the TTM EPS and P/E ratios:
    - Initialize 'EPS (ttm)' and 'P/E (ttm)' columns with NaN values.
    - For each row, if there are enough previous rows (189 days), calculate the TTM EPS as the sum of earnings 
      from the current row and the rows 63, 126, and 189 days prior.
    - Calculate the TTM P/E ratio as the adjusted close price divided by the TTM EPS.
5. Concatenate the processed groups and write the final DataFrame to 'stocks_calc_final_with_ttm.csv'.
Output:
- 'stocks_calc_final_with_ttm.csv': A CSV file containing the original data along with the calculated 'EPS (ttm)' 
  and 'P/E (ttm)' columns.

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("stocks_calc_final.csv")
df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
df = df.sort_values(["Symbol", "Date"])

output_groups = []

for symbol, group in df.groupby("Symbol", group_keys=False):
    # Sort by date, reset index so each row has a clean 0..N index
    group = group.sort_values("Date").reset_index(drop=True)
    
    # Initialize columns
    group["EPS (ttm)"] = np.nan
    group["P/E (ttm)"] = np.nan
    
    for i in range(len(group)):
        if i >= 189:  # Enough rows
            earnings_values = [
                group.at[i, "Earnings"],
                group.at[i-63, "Earnings"],
                group.at[i-126, "Earnings"],
                group.at[i-189, "Earnings"]
            ]
            if all(pd.notna(earnings_values)) and all(earnings_values):
                eps_ttm = sum(earnings_values)
                group.at[i, "EPS (ttm)"] = eps_ttm
                if eps_ttm != 0:
                    group.at[i, "P/E (ttm)"] = group.at[i, "Adjusted Close"] / eps_ttm
    
    output_groups.append(group)

final_df = pd.concat(output_groups, ignore_index=True)
final_df.to_csv("stocks_calc_final_with_ttm.csv", index=False)

In [1]:
import pandas as pd
import numpy as np

# Read the main stock data file
print("Loading main stock data...")
stock_df = pd.read_csv("../stocks_calc_final_with_ttm.csv")
stock_df["Date"] = pd.to_datetime(stock_df["Date"], errors="coerce")

# Load economic indicator data files
print("Loading economic indicator data...")

# CPI data
cpi_df = pd.read_csv("../CPI.csv")
cpi_df["Date"] = pd.to_datetime(cpi_df["Date"], errors="coerce")
cpi_df.rename(columns={"Value": "CPI"}, inplace=True)

# Federal Funds Rate data
ffr_df = pd.read_csv("../FFR.csv")
ffr_df["Date"] = pd.to_datetime(ffr_df["Date"], errors="coerce")
ffr_df.rename(columns={"Value": "Fed_Funds_Rate"}, inplace=True)

# GDP Per Capita data
gdp_capita_df = pd.read_csv("../GDP_Capita.csv")
gdp_capita_df["Date"] = pd.to_datetime(gdp_capita_df["Date"], errors="coerce")
gdp_capita_df.rename(columns={"Value": "GDP_Per_Capita"}, inplace=True)

# Inflation data
inflation_df = pd.read_csv("../INFLATION.csv")
inflation_df["Date"] = pd.to_datetime(inflation_df["Date"], errors="coerce")
inflation_df.rename(columns={"Value": "Inflation_Rate"}, inplace=True)

# Real GDP data
real_gdp_df = pd.read_csv("../REAL_GDP.csv")
real_gdp_df["Date"] = pd.to_datetime(real_gdp_df["Date"], errors="coerce")
real_gdp_df.rename(columns={"Value": "Real_GDP"}, inplace=True)

# Treasury Yield data
treasury_df = pd.read_csv("../TREASURY_YIELD.csv")
treasury_df["Date"] = pd.to_datetime(treasury_df["Date"], errors="coerce")
treasury_df.rename(columns={"Value": "Treasury_Yield"}, inplace=True)

# Unemployment data
unemployment_df = pd.read_csv("../UNEMPLOYMENT.csv")
unemployment_df["Date"] = pd.to_datetime(unemployment_df["Date"], errors="coerce")
unemployment_df.rename(columns={"Value": "Unemployment_Rate"}, inplace=True)

# Create a function to resample monthly data to daily with forward fill
def resample_to_daily(df):
    # Set Date as index
    df = df.set_index('Date')
    
    # Create a new date range at daily frequency from min to max date
    date_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq='D')
    
    # Reindex to have all days, and forward fill missing values
    df_daily = df.reindex(date_range).ffill()
    
    # Reset index to get Date as a column again
    df_daily = df_daily.reset_index()
    df_daily.rename(columns={'index': 'Date'}, inplace=True)
    return df_daily
# Resample each economic indicator DataFrame to daily frequency
cpi_daily = resample_to_daily(cpi_df)
ffr_daily = resample_to_daily(ffr_df)
gdp_capita_daily = resample_to_daily(gdp_capita_df)
inflation_daily = resample_to_daily(inflation_df)
real_gdp_daily = resample_to_daily(real_gdp_df)
treasury_daily = resample_to_daily(treasury_df)
unemployment_daily = resample_to_daily(unemployment_df)
# Merge all economic indicators with the main stock DataFrame
print("Merging economic indicators with stock data...")
merged_df = stock_df.merge(cpi_daily, on="Date", how="left")
merged_df = merged_df.merge(ffr_daily, on="Date", how="left")
merged_df = merged_df.merge(gdp_capita_daily, on="Date", how="left")
merged_df = merged_df.merge(inflation_daily, on="Date", how="left")
merged_df = merged_df.merge(real_gdp_daily, on="Date", how="left")
merged_df = merged_df.merge(treasury_daily, on="Date", how="left")
merged_df = merged_df.merge(unemployment_daily, on="Date", how="left")
# Save the final merged DataFrame to CSV
print("Saving final merged data to CSV...")
merged_df.to_csv("stocks_final_with_economic_indicators.csv", index=False)
# Display the first few rows of the final DataFrame
print(merged_df.head())

Loading main stock data...
Loading economic indicator data...
Merging economic indicators with stock data...
Saving final merged data to CSV...
  Symbol       Date   Open   High    Low  Close  Adjusted Close    Volume  \
0      A 1999-11-18  45.50  50.00  40.00  44.00       28.452814  44739900   
1      A 1999-11-19  42.94  43.00  39.81  40.38       26.111923  10897100   
2      A 1999-11-22  41.31  44.00  40.06  44.00       28.452814   4705200   
3      A 1999-11-23  42.50  43.63  40.25  40.25       26.027858   4274400   
4      A 1999-11-24  40.13  41.94  40.00  41.06       26.551649   3464400   

   Dividend Amount  Split Coefficient  ...  P/E Ratio  EPS (ttm)  P/E (ttm)  \
0              0.0                1.0  ...        NaN        NaN        NaN   
1              0.0                1.0  ...        NaN        NaN        NaN   
2              0.0                1.0  ...        NaN        NaN        NaN   
3              0.0                1.0  ...        NaN        NaN        NaN  

Create new columns

In [2]:
import pandas as pd

# Read the CSV
df = pd.read_csv("/Users/daniellott2/Documents/stock-program/src/data/stocks_final_with_economic_indicators.csv")

# Ensure Date is recognized as a datetime for proper sorting
df["Date"] = pd.to_datetime(df["Date"], errors="coerce")

# Sort by symbol and date
df = df.sort_values(["Symbol", "Date"])

# Calculate moving averages
df["MA_50"] = df.groupby("Symbol")["Adjusted Close"].transform(lambda x: x.rolling(window=50).mean())
df["MA_200"] = df.groupby("Symbol")["Adjusted Close"].transform(lambda x: x.rolling(window=200).mean())

# Drop rows with any N/A values after moving averages calculation
df = df.dropna(subset=["Adjusted Close", "MA_50", "MA_200"])  # Only drop if key values are missing

# Calculate price change over 1, 5, and 10 years
df["Price_Change_1Y"] = df.groupby("Symbol")["Adjusted Close"].transform(lambda x: x.pct_change(periods=252))
df["Price_Change_5Y"] = df.groupby("Symbol")["Adjusted Close"].transform(lambda x: x.pct_change(periods=1260))
df["Price_Change_10Y"] = df.groupby("Symbol")["Adjusted Close"].transform(lambda x: x.pct_change(periods=2520))

df["MA_50/Adj_Close"] = df["MA_50"] / df["Adjusted Close"]
df["MA_200/Adj_Close"] = df["MA_200"] / df["Adjusted Close"]

# Drop rows with any N/A values in the respective columns and save to different files
df_1yr = df.dropna(subset=["Price_Change_1Y"])
df_1yr.to_csv("../stocks_with_1yr_price_change2.csv", index=False)

df_5yr = df.dropna(subset=["Price_Change_5Y"])
df_5yr.to_csv("../stocks_with_5yr_price_change2.csv", index=False)

df_10yr = df.dropna(subset=["Price_Change_10Y"])
df_10yr.to_csv("../stocks_with_10yr_price_change2.csv", index=False)

# fix survivorship bias

In [1]:
# (macro columns kept, everything else NaN after delist) ----
import pandas as pd
from pathlib import Path
from pandas.tseries.offsets import BDay

DATA_DIR   = Path("data")
INPUT_CSV  = DATA_DIR / "stocks_with_1yr_price_change2.csv"
OUTPUT_CSV = DATA_DIR / "stocks_cleaned_final.csv"
END_DATE   = "2024-10-15"

PRICE_COLS = ["Open", "High", "Low", "Close", "Adjusted Close"]
MACRO_COLS = ["CPI", "Fed_Funds_Rate", "GDP_Per_Capita",
              "Inflation_Rate", "Real_GDP", "Treasury_Yield",
              "Unemployment_Rate"]                       # keep these
# ------------------------------------------------------------------------------

def clean_delistings():
    df = pd.read_csv(INPUT_CSV, parse_dates=["Date"])
    df.sort_values(["Symbol", "Date"], inplace=True)

    calendar = pd.date_range(df["Date"].min(), END_DATE, freq="B")
    frames = []

    for sym, g in df.groupby("Symbol"):
        g = g.set_index("Date").reindex(calendar)
        g["Symbol"] = sym

        last_trade = g.dropna(subset=["Adjusted Close"]).index.max()

        # 1) Prices: flat‑line to END_DATE
        g[PRICE_COLS] = g[PRICE_COLS].ffill()

        # 2) Volume: zero past delist
        g["Volume"] = g["Volume"].ffill()
        g.loc[g.index > last_trade, "Volume"] = 0

        # 3) Company‑specific fundamentals -> NaN past delist
        company_cols = [c for c in g.columns
                        if c not in PRICE_COLS + ["Volume", "Symbol", "Date"] + MACRO_COLS]
        g.loc[g.index > last_trade, company_cols] = pd.NA

        # (Macro columns left untouched — they're global)
        frames.append(g.reset_index().rename(columns={"index": "Date"}))

    cleaned = pd.concat(frames, ignore_index=True)
    cleaned.to_csv(OUTPUT_CSV, index=False)
    print(f"Saved cleaned file → {OUTPUT_CSV}")

if __name__ == "__main__":
    clean_delistings()


Saved cleaned file → data/stocks_cleaned_final.csv


# add future change columns

In [3]:
import pandas as pd
import numpy as np
from pathlib import Path

# Read the cleaned data
input_path = Path("data") / "stocks_cleaned_final.csv"
df = pd.read_csv(input_path, parse_dates=["Date"])

# Sort by symbol and date
df = df.sort_values(["Symbol", "Date"])

# Create columns for future price changes (1Y, 3Y, 5Y, 10Y)
df["Future_Price_Change_1Y"] = np.nan  # Initialize with NaN
df["Future_Price_Change_3Y"] = np.nan  # Initialize with NaN
df["Future_Price_Change_5Y"] = np.nan  # Initialize with NaN
df["Future_Price_Change_10Y"] = np.nan  # Initialize with NaN

# Define trading days for different time periods
DAYS_1Y = 252
DAYS_3Y = 756    # 252 * 3
DAYS_5Y = 1260   # 252 * 5
DAYS_10Y = 2520  # 252 * 10

# Process each stock separately
for symbol, group in df.groupby("Symbol"):
    # Sort by date
    group = group.sort_values("Date")
    
    # Get the Adjusted Close values and indices
    adj_close = group["Adjusted Close"].values
    group_indices = group.index.tolist()
    
    # Calculate 1-year future returns
    for i in range(len(group) - DAYS_1Y):
        current_price = adj_close[i]
        future_price = adj_close[i + DAYS_1Y]
        
        if current_price > 0:  # Avoid division by zero
            future_change = (future_price - current_price) / current_price
            df.loc[group_indices[i], "Future_Price_Change_1Y"] = future_change
    
    # Calculate 3-year future returns
    for i in range(len(group) - DAYS_3Y):
        current_price = adj_close[i]
        future_price = adj_close[i + DAYS_3Y]
        
        if current_price > 0:
            future_change = (future_price - current_price) / current_price
            df.loc[group_indices[i], "Future_Price_Change_3Y"] = future_change
    
    # Calculate 5-year future returns
    for i in range(len(group) - DAYS_5Y):
        current_price = adj_close[i]
        future_price = adj_close[i + DAYS_5Y]
        
        if current_price > 0:
            future_change = (future_price - current_price) / current_price
            df.loc[group_indices[i], "Future_Price_Change_5Y"] = future_change
    
    # Calculate 10-year future returns
    for i in range(len(group) - DAYS_10Y):
        current_price = adj_close[i]
        future_price = adj_close[i + DAYS_10Y]
        
        if current_price > 0:
            future_change = (future_price - current_price) / current_price
            df.loc[group_indices[i], "Future_Price_Change_10Y"] = future_change

# Save the result
output_path = Path("data") / "stocks_final.csv"
df.to_csv(output_path, index=False)
print(f"Saved file with future returns → {output_path}")

Saved file with future returns → data/stocks_final.csv
